# Risk Calculator 

This notebook calculates game-by-game injury risk using rolling performance, fatigue indicators, and a Bayesian-style volatility measure, based on the FanGraphs method linked here: https://community.fangraphs.com/projecting-risk-in-major-league-baseball-a-bayesian-approach/ 

## Imports 

In [7]:
import sqlite3
import pandas as pd
import numpy as np
import os
import sys
import argparse
import logging

## Connection to Database


In [3]:
# Connect to your SQLite database
db_path = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/data/dodgers_injury_db.sqlite'
conn = sqlite3.connect(db_path)

# Load the game_logs table into a DataFrame
df = pd.read_sql_query("SELECT * FROM game_logs", conn)


## Data Processing

In [4]:
# Ensure proper date format and sort
df['game_date'] = pd.to_datetime(df['game_date'])
df = df.sort_values(by=['mlb_player_id', 'game_date'])

# Add Plate Appearances
df['pa'] = df['ab'] + df['bb'] + df['hbp'].fillna(0) + df['sf'].fillna(0)

# Calculate rolling hits and ABs
df['rolling_hits'] = df.groupby('mlb_player_id')['h'].transform(lambda x: x.rolling(5, min_periods=1).sum())
df['rolling_abs'] = df.groupby('mlb_player_id')['ab'].transform(lambda x: x.rolling(5, min_periods=1).sum())
df['avg_5g'] = df['rolling_hits'] / df['rolling_abs']

## The Math of Calculating Risk

In [5]:
# Back-to-back fatigue
df['days_since_last'] = df.groupby('mlb_player_id')['game_date'].diff().dt.days
df['back_to_back'] = df['days_since_last'] == 1

# 7-day total bases
df['tb_7d'] = df.groupby('mlb_player_id')['tb'].transform(
    lambda x: x.rolling(window=7, min_periods=1).sum()
)

In [8]:
# Hit rate and TB per hit
df['hit_rate'] = df['h'] / df['ab'].replace(0, np.nan)
df['tb_per_hit'] = df['tb'] / df['h'].replace(0, np.nan)

# Expected TB using hit rate
df['expected_tb'] = df['hit_rate'] * df['ab'] * df['tb_per_hit']

# Rolling volatility in total bases
df['tb_volatility'] = df.groupby('mlb_player_id')['tb'].transform(
    lambda x: x.rolling(window=7, min_periods=3).std()
)

# Risk-adjusted total bases (like Sharpe Ratio)
df['risk_adjusted_tb'] = (
    (df['tb'] - df['expected_tb']) / df['tb_volatility'].replace(0, np.nan)
)

In [9]:
df['risk_score'] = (
    df['back_to_back'].astype(int) +
    (df['tb_7d'] > 20).astype(int) +
    (df['avg_5g'] < 0.2).astype(int) +
    (df['risk_adjusted_tb'] < -1).astype(int)
)

## The risk Table 

In [ ]:
# Format final table
risk_table = df[[
    'mlb_player_id', 'game_date', 'team', 'opponent',
    'ab', 'h', 'tb', 'avg_5g', 'tb_7d', 'risk_adjusted_tb',
    'back_to_back', 'risk_score'
]].rename(columns={
    'mlb_player_id': 'Player ID',
    'game_date': 'Game Date',
    'team': 'Team',
    'opponent': 'Opponent',
    'ab': 'At Bats',
    'h': 'Hits',
    'tb': 'Total Bases',
    'avg_5g': '5-Game AVG',
    'tb_7d': '7-Day TB',
    'risk_adjusted_tb': 'Risk-Adj TB',
    'back_to_back': 'Back-to-Back Game',
    'risk_score': 'Risk Score'
})

# Save or preview

risk_table.head(10)

,Player ID,Game Date,Team,Opponent,At Bats,Hits,Total Bases,5-Game AVG,7-Day TB,Risk-Adj TB,Back-to-Back Game,Risk Score
0,472610,2025-03-19,Los Angeles Dodgers,Chicago Cubs,5,2,2,0.400000,2.0,NaN,False,0
1,472610,2025-03-28,Los Angeles Dodgers,Detroit Tigers,4,1,3,0.333333,5.0,NaN,False,0
2,472610,2025-03-29,Los Angeles Dodgers,Detroit Tigers,7,2,4,0.312500,9.0,0.0,True,1
3,472610,2025-03-31,Los Angeles Dodgers,Atlanta Braves,4,1,1,0.300000,10.0,0.0,False,0
4,472610,2025-04-04,Los Angeles Dodgers,Philadelphia Phillies,4,0,0,0.250000,10.0,NaN,False,0
5,472610,2025-04-06,Los Angeles Dodgers,Philadelphia Phillies,2,0,0,0.190476,10.0,NaN,False,1
6,472610,2025-04-08,Los Angeles Dodgers,Washington Nationals,4,1,1,0.190476,11.0,0.0,False,1
7,472610,2025-04-12,Los Angeles Dodgers,Chicago Cubs,4,3,7,0.277778,16.0,0.0,False,0
8,472610,2025-04-15,Los Angeles Dodgers,Colorado Rockies,3,0,0,0.235294,13.0,NaN,False,0
9,472610,2025-04-20,Los Angeles Dodgers,Texas Rangers,2,0,0,0.266667,9.0,NaN,False,0


In [ ]:
# Preview
risk_table.head(10)